### Correction 1 (run this) for XAUUSDm, BTCUSDm (15min)

- Changed TP/SL to be lowest/highest of previous 2 candles + 10 pips
- Trailing SL changed for SL to move to entry price when Price move towards TP

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import talib
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "BTCUSDm"
TIMEFRAME = mt5.TIMEFRAME_M5
timeframe_minutes = 5
LOT_SIZE = 0.07
MAGIC_NUMBER = 10004
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 1000  # 💰 Target profit in USD
INITIAL_BALANCE = None  # 🔒 Will be set after login
stop_trading = False  # Shared flag

# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=242864620,
        password="Password1234.",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()
        
    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

# === (Rest of your existing code remains unchanged — insert check below into main loop) ===

def wait_for_next_candle():
    now = datetime.now()
    seconds_in_timeframe = timeframe_minutes * 60
    seconds_since_epoch = int(now.timestamp())
    seconds_to_next_candle = seconds_in_timeframe - (seconds_since_epoch % seconds_in_timeframe)
    # print(f"⏳ Waiting {seconds_to_next_candle}s for next candle...")
    time.sleep(seconds_to_next_candle)

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# == AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(df, entry_price, signal):
    pip_value = 1
    buffer = 20 * pip_value  # 300 pips buffer

    # Get the previous two candles
    prev_2 = df.iloc[-3]
    prev_1 = df.iloc[-2]
    

    if signal == "BUY":
        lowest_low = min(prev_1['low'], prev_2['low'])
        stop_loss = lowest_low - buffer
        take_profit = entry_price + 3 * (entry_price - stop_loss)

    elif signal == "SELL":
        highest_high = max(prev_1['high'], prev_2['high'])
        stop_loss = highest_high + buffer
        take_profit = entry_price - 3 * (stop_loss - entry_price)

    else:
        stop_loss = None
        take_profit = None

    return stop_loss, take_profit

# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === IDENTIFY TRADE SIGNALS === SHOOTING STAR, HAMMER, DOJI
def generate_signals(df):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None

    if len(df) < 6:
        return signal, entry_price, stop_loss, take_profit

    # Candle pattern indicators
    df['hammer'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
    df['inverted_hammer'] = talib.CDLINVERTEDHAMMER(df['open'], df['high'], df['low'], df['close'])
    df['hanging_man'] = talib.CDLHANGINGMAN(df['open'], df['high'], df['low'], df['close'])
    df['shooting_star'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
    df['dragonfly_doji'] = talib.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close'])
    df['gravestone_doji'] = talib.CDLGRAVESTONEDOJI(df['open'], df['high'], df['low'], df['close'])
    df['long_legged_doji'] = talib.CDLLONGLEGGEDDOJI(df['open'], df['high'], df['low'], df['close'])
    df['marubozu'] = talib.CDLMARUBOZU(df['open'], df['high'], df['low'], df['close'])
    df['spinning_top'] = talib.CDLSPINNINGTOP(df['open'], df['high'], df['low'], df['close'])

    # Previous 3 candles
    prev_3 = df.iloc[-5]
    prev_2 = df.iloc[-4]
    prev_1 = df.iloc[-3]

    # Signal candle
    signal_candle = df.iloc[-2]

    # Trend setup
    three_bearish = all([c['close'] < c['open'] for c in [prev_3, prev_2, prev_1]])
    three_bullish = all([c['close'] > c['open'] for c in [prev_3, prev_2, prev_1]])

    # Define bullish signal conditions
    bullish_patterns = [
        signal_candle['hammer'],
        signal_candle['inverted_hammer'],
        signal_candle['dragonfly_doji'],
        signal_candle['long_legged_doji'],
        signal_candle['spinning_top'],
        signal_candle['marubozu']
    ]

    # Define bearish signal conditions
    bearish_patterns = [
        signal_candle['shooting_star'],
        signal_candle['hanging_man'],
        signal_candle['gravestone_doji'],
        signal_candle['long_legged_doji'],
        signal_candle['spinning_top'],
        signal_candle['marubozu']
    ]

    # BUY signal
    if three_bearish and any(p > 0 for p in bullish_patterns):
        signal = "BUY"
        entry_price = df.iloc[-1]['open']

    # SELL signal
    elif three_bullish and any(p < 0 for p in bearish_patterns):
        signal = "SELL"
        entry_price = df.iloc[-1]['open']

    if signal:
        stop_loss, take_profit = predict_sl_tp(df, entry_price, signal)

    return signal, entry_price, stop_loss, take_profit

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:  # Ensure the order is not None
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: {entry_price}")
            position_id = order.order
            time.sleep(2)
            modify_request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": position_id,
                "sl": stop_loss,
                "tp": take_profit,
                "magic": MAGIC_NUMBER
            }
            modify = mt5.order_send(modify_request)
            if modify.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ SL/TP Updated: SL={stop_loss}, TP={take_profit}")
            else:
                print(f"❌ Failed to update SL/TP: {modify.retcode}")
        else:
            print(f"❌ Trade Failed: {order.retcode}")
    else:
        print("❌ Order send failed: order is None")
        
def equity_check_and_close_trades():
    global stop_trading, INITIAL_BALANCE, PROFIT_TARGET
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return

    target_equity = INITIAL_BALANCE + PROFIT_TARGET
    print(f"🎯 Monitoring for equity to reach ${target_equity:.2f}...")

    while not stop_trading:
        account_info = mt5.account_info()
        if account_info is None:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")
            wait_for_next_candle()
            continue

        equity = account_info.equity
        # print(f"🔍 Current Equity: ${equity:.2f} | Target: ${target_equity:.2f}")

        if equity >= target_equity:
            print("🚀 Equity target reached! Closing all open positions...")
            positions = mt5.positions_get()
            if positions is None or len(positions) == 0:
                print("No open positions to close.")
            else:
                for pos in positions:
                    symbol = pos.symbol
                    volume = pos.volume
                    ticket = pos.ticket
                    order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                    price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

                    close_request = {
                        "action": mt5.TRADE_ACTION_DEAL,
                        "position": ticket,
                        "symbol": symbol,
                        "volume": volume,
                        "type": order_type,
                        "price": price,
                        "deviation": 10,
                        "magic": 123456,
                        "comment": "Auto close on equity profit"
                    }

                    result = mt5.order_send(close_request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"✅ Position {ticket} closed successfully.")
                    else:
                        print(f"❌ Failed to close position {ticket}: {result.retcode}")

            stop_trading = True  # Set the stop flag
            break

        wait_for_next_candle()

# === RUN THE BOT ===
def run_bot():
    global stop_trading
    connect_mt5()

    # Start the equity check in a separate thread
    threading.Thread(target=equity_check_and_close_trades, daemon=True).start()
    
    while not stop_trading:
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            wait_for_next_candle()
            continue

        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        # ✅ Always check and update SL to breakeven for open positions
        positions = mt5.positions_get(symbol=SYMBOL)
        if positions:
            tick = mt5.symbol_info_tick(SYMBOL)
            if tick:
                for pos in positions:
                    entry = pos.price_open
                    current_sl = pos.sl

                    if pos.type == mt5.ORDER_TYPE_BUY:
                        current_price = tick.ask
                        if current_price > entry and current_sl < entry:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": round(entry, 5),
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ BUY SL moved to breakeven for position {pos.ticket}")
                            else:
                                print(f"❌ Failed to update BUY SL: {result.retcode}")

                    elif pos.type == mt5.ORDER_TYPE_SELL:
                        current_price = tick.bid
                        if current_price < entry and current_sl > entry:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": round(entry, 5),
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ SELL SL moved to breakeven for position {pos.ticket}")
                            else:
                                print(f"❌ Failed to update SELL SL: {result.retcode}")
            else:
                print("❌ Failed to fetch current price tick.")

        # ✅ Trade only if signal exists and allowed
        if signal:
            allow_trade = True
            if positions:
                for pos in positions:
                    if pos.type == mt5.ORDER_TYPE_BUY and signal == "BUY":
                        print("⚠️ Trade skipped: Already in a BUY position.")
                        allow_trade = False
                        break
                    elif pos.type == mt5.ORDER_TYPE_SELL and signal == "SELL":
                        print("⚠️ Trade skipped: Already in a SELL position.")
                        allow_trade = False
                        break

            if allow_trade:
                print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
                place_trade(signal, entry_price, stop_loss, take_profit)
            else:
                print("⏳ Waiting for new opposite signal...")
        # else:
        #     print("⏳ No trade opportunity found.")
    
        wait_for_next_candle()
    
    print("🛑 Trading stopped due to equity target being reached.")

# Start the trading bot
run_bot()


✅ MT5 initialized and logged in successfully.
✅ Initial Balance: 1733.14
🎯 Monitoring for equity to reach $2233.14...


### BTC 5min

- Added Max and Min stop losses *This is to keep stop loss always at a max GHC100 at 0.1 lots size
- Stop loss moves to break even when price is Above entry
- Stop loss determine with predetermined pips

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import talib
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "BTCUSDm"
TIMEFRAME = mt5.TIMEFRAME_M5
timeframe_minutes = 5
LOT_SIZE = 0.05
MAGIC_NUMBER = 10004
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 1000  # 💰 Target profit in USD
INITIAL_BALANCE = None  # 🔒 Will be set after login
stop_trading = False  # Shared flag

# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=242864620,
        password="Password1234.",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()
        
    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

# === (Rest of your existing code remains unchanged — insert check below into main loop) ===

def wait_for_next_candle():
    now = datetime.now()
    seconds_in_timeframe = timeframe_minutes * 60
    seconds_since_epoch = int(now.timestamp())
    seconds_to_next_candle = seconds_in_timeframe - (seconds_since_epoch % seconds_in_timeframe)
    # print(f"⏳ Waiting {seconds_to_next_candle}s for next candle...")
    time.sleep(seconds_to_next_candle)

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# == AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(df, entry_price, signal):
    pip_value = 1
    buffer = 10 * pip_value

    min_sl_pips = 120 * pip_value   # 70
    max_sl_pips = 200 * pip_value   # 100

    prev_2 = df.iloc[-3]
    prev_1 = df.iloc[-2]

    if signal == "BUY":
        lowest_low = min(prev_1['low'], prev_2['low'])
        raw_stop_loss = lowest_low - buffer
        max_allowed_sl = entry_price - min_sl_pips
        min_allowed_sl = entry_price - max_sl_pips
        stop_loss = max(min(raw_stop_loss, max_allowed_sl), min_allowed_sl)
        take_profit = entry_price + 2 * (entry_price - stop_loss)

    elif signal == "SELL":
        highest_high = max(prev_1['high'], prev_2['high'])
        raw_stop_loss = highest_high + buffer
        min_allowed_sl = entry_price + min_sl_pips
        max_allowed_sl = entry_price + max_sl_pips
        stop_loss = min(max(raw_stop_loss, min_allowed_sl), max_allowed_sl)
        take_profit = entry_price - 2 * (stop_loss - entry_price)

    else:
        stop_loss = None
        take_profit = None

    return stop_loss, take_profit

# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === IDENTIFY TRADE SIGNALS === SHOOTING STAR, HAMMER, DOJI
def generate_signals(df):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None

    if len(df) < 6:
        return signal, entry_price, stop_loss, take_profit

    # Candle pattern indicators
    df['hammer'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
    df['inverted_hammer'] = talib.CDLINVERTEDHAMMER(df['open'], df['high'], df['low'], df['close'])
    df['hanging_man'] = talib.CDLHANGINGMAN(df['open'], df['high'], df['low'], df['close'])
    df['shooting_star'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
    df['dragonfly_doji'] = talib.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close'])
    df['gravestone_doji'] = talib.CDLGRAVESTONEDOJI(df['open'], df['high'], df['low'], df['close'])
    df['long_legged_doji'] = talib.CDLLONGLEGGEDDOJI(df['open'], df['high'], df['low'], df['close'])
    df['marubozu'] = talib.CDLMARUBOZU(df['open'], df['high'], df['low'], df['close'])
    df['spinning_top'] = talib.CDLSPINNINGTOP(df['open'], df['high'], df['low'], df['close'])
    df['takuri'] = talib.CDLTAKURI(df['open'], df['high'], df['low'], df['close'])
    df['rickshawman'] = talib.CDLRICKSHAWMAN(df['open'], df['high'], df['low'], df['close'])
    df['engulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])

    # Previous 3 candles
    prev_3 = df.iloc[-5]
    prev_2 = df.iloc[-4]
    prev_1 = df.iloc[-3]

    # Signal candle
    signal_candle = df.iloc[-2]

    # Trend setup
    three_bearish = all([c['close'] < c['open'] for c in [prev_2, prev_1]])
    three_bullish = all([c['close'] > c['open'] for c in [prev_2, prev_1]])

    # Define bullish signal conditions
    bullish_patterns = [
        signal_candle['hammer'],
        signal_candle['inverted_hammer'],
        signal_candle['dragonfly_doji'],
        signal_candle['long_legged_doji'],
        signal_candle['spinning_top'],
        signal_candle['takuri'],
        signal_candle['rickshawman'],
        signal_candle['shooting_star'],
        signal_candle['hanging_man'],
        signal_candle['gravestone_doji'],
        signal_candle['engulfing']
    ]

    # Define bearish signal conditions
    bearish_patterns = [
        signal_candle['shooting_star'],
        signal_candle['hanging_man'],
        signal_candle['gravestone_doji'],
        signal_candle['long_legged_doji'],
        signal_candle['spinning_top'],
        signal_candle['rickshawman'],
        signal_candle['hammer'],
        signal_candle['inverted_hammer'],
        signal_candle['takuri'],
        signal_candle['dragonfly_doji'],
        signal_candle['engulfing']
    ]

    # BUY signal
    if three_bearish and any(p > 0 for p in bullish_patterns):
        signal = "BUY"
        entry_price = df.iloc[-1]['open']

    # SELL signal
    elif three_bullish and any(p < 0 for p in bearish_patterns):
        signal = "SELL"
        entry_price = df.iloc[-1]['open']

    if signal:
        stop_loss, take_profit = predict_sl_tp(df, entry_price, signal)

    return signal, entry_price, stop_loss, take_profit

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:  # Ensure the order is not None
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: {entry_price}")
            position_id = order.order
            time.sleep(2)
            modify_request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": position_id,
                "sl": stop_loss,
                "tp": take_profit,
                "magic": MAGIC_NUMBER
            }
            modify = mt5.order_send(modify_request)
            if modify.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ SL/TP Updated: SL={stop_loss}, TP={take_profit}")
            else:
                print(f"❌ Failed to update SL/TP: {modify.retcode}")
        else:
            print(f"❌ Trade Failed: {order.retcode}")
    else:
        print("❌ Order send failed: order is None")
        
def equity_check_and_close_trades():
    global stop_trading, INITIAL_BALANCE, PROFIT_TARGET
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return

    target_equity = INITIAL_BALANCE + PROFIT_TARGET
    print(f"🎯 Monitoring for equity to reach ${target_equity:.2f}...")

    while not stop_trading:
        account_info = mt5.account_info()
        if account_info is None:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")
            wait_for_next_candle()
            continue

        equity = account_info.equity
        # print(f"🔍 Current Equity: ${equity:.2f} | Target: ${target_equity:.2f}")

        if equity >= target_equity:
            print("🚀 Equity target reached! Closing all open positions...")
            positions = mt5.positions_get()
            if positions is None or len(positions) == 0:
                print("No open positions to close.")
            else:
                for pos in positions:
                    symbol = pos.symbol
                    volume = pos.volume
                    ticket = pos.ticket
                    order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                    price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

                    close_request = {
                        "action": mt5.TRADE_ACTION_DEAL,
                        "position": ticket,
                        "symbol": symbol,
                        "volume": volume,
                        "type": order_type,
                        "price": price,
                        "deviation": 10,
                        "magic": 123456,
                        "comment": "Auto close on equity profit"
                    }

                    result = mt5.order_send(close_request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"✅ Position {ticket} closed successfully.")
                    else:
                        print(f"❌ Failed to close position {ticket}: {result.retcode}")

            stop_trading = True  # Set the stop flag
            break

        wait_for_next_candle()

def stop_at_6am_gmt():
    global stop_trading
    print("🕕 Thread to monitor 6:00 AM GMT started...")
    while not stop_trading:
        now = datetime.utcnow()
        if now.hour == 6 and now.minute == 0:
            print("🛑 It's 6:00 AM GMT! Closing all trades and stopping the bot.")
            positions = mt5.positions_get()
            if positions:
                for pos in positions:
                    symbol = pos.symbol
                    volume = pos.volume
                    ticket = pos.ticket
                    order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                    price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

                    close_request = {
                        "action": mt5.TRADE_ACTION_DEAL,
                        "position": ticket,
                        "symbol": symbol,
                        "volume": volume,
                        "type": order_type,
                        "price": price,
                        "deviation": 10,
                        "magic": MAGIC_NUMBER,
                        "comment": "Auto close at 6AM GMT"
                    }

                    result = mt5.order_send(close_request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"✅ Position {ticket} closed successfully at 6AM GMT.")
                    else:
                        print(f"❌ Failed to close position {ticket}: {result.retcode}")
            else:
                print("ℹ️ No open positions to close at 6AM GMT.")

            stop_trading = True
            break

        time.sleep(10)  # check every 10 seconds

# === RUN THE BOT ===
def run_bot():
    global stop_trading
    connect_mt5()

    # Start the equity check in a separate thread
    threading.Thread(target=equity_check_and_close_trades, daemon=True).start()
    threading.Thread(target=stop_at_6am_gmt, daemon=True).start()
    
    while not stop_trading:
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            wait_for_next_candle()
            continue

        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        # ✅ Always check and update SL to breakeven for open positions
        positions = mt5.positions_get(symbol=SYMBOL)
        if positions:
            tick = mt5.symbol_info_tick(SYMBOL)
            if tick:
                for pos in positions:
                    entry = pos.price_open
                    current_sl = pos.sl
                    current_tp = pos.tp
                    sl_distance = abs(entry - current_sl)
                    
                    if pos.type == mt5.ORDER_TYPE_BUY:
                        current_price = tick.ask
                        if current_price > entry and current_sl < entry:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": round(entry, 5),
                                "tp": current_tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ BUY SL moved to breakeven for position {pos.ticket}")
                            else:
                                print(f"❌ Failed to update BUY SL: {result.retcode}")

                    elif pos.type == mt5.ORDER_TYPE_SELL:
                        current_price = tick.bid
                        if current_price <= entry - 1.5 * sl_distance and current_sl > entry:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": round(entry, 5),
                                "tp": current_tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ SELL SL moved to breakeven for position {pos.ticket}")
                            else:
                                print(f"❌ Failed to update SELL SL: {result.retcode}")
            else:
                print("❌ Failed to fetch current price tick.")
                
        # ✅ Trade only if signal exists and allowed
        if signal:
            allow_trade = True
            if positions:
                for pos in positions:
                    if pos.type == mt5.ORDER_TYPE_BUY and signal == "BUY":
                        print("⚠️ Trade skipped: Already in a BUY position.")
                        allow_trade = False
                        break
                    elif pos.type == mt5.ORDER_TYPE_SELL and signal == "SELL":
                        print("⚠️ Trade skipped: Already in a SELL position.")
                        allow_trade = False
                        break

            if allow_trade:
                print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
                place_trade(signal, entry_price, stop_loss, take_profit)
            # else:
            #     print("⏳ Waiting for new opposite signal...")
        # else:
        #     print("⏳ No trade opportunity found.")
    
        wait_for_next_candle()
    
    print("🛑 Trading stopped due to equity target being reached.")

# Start the trading bot
run_bot()


### BTC with ATR

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import talib
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "BTCUSDm"
TIMEFRAME = mt5.TIMEFRAME_M5
timeframe_minutes = 5
LOT_SIZE = 0.05
MAGIC_NUMBER = 10005
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 1000  # 💰 Target profit in USD
INITIAL_BALANCE = None  # 🔒 Will be set after login
stop_trading = False  # Shared flag

# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=242864620,
        password="Password1234.",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()
        
    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

# === (Rest of your existing code remains unchanged — insert check below into main loop) ===

def wait_for_next_candle():
    now = datetime.now()
    seconds_in_timeframe = timeframe_minutes * 60
    seconds_since_epoch = int(now.timestamp())
    seconds_to_next_candle = seconds_in_timeframe - (seconds_since_epoch % seconds_in_timeframe)
    # print(f"⏳ Waiting {seconds_to_next_candle}s for next candle...")
    time.sleep(seconds_to_next_candle)

# === IDENTIFY TRADE SIGNALS === SHOOTING STAR, HAMMER, DOJI
def generate_signals(df):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None

    if len(df) < 6:
        return signal, entry_price, stop_loss, take_profit

    # Candle pattern indicators
    df['hammer'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
    df['inverted_hammer'] = talib.CDLINVERTEDHAMMER(df['open'], df['high'], df['low'], df['close'])
    df['hanging_man'] = talib.CDLHANGINGMAN(df['open'], df['high'], df['low'], df['close'])
    df['shooting_star'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
    df['dragonfly_doji'] = talib.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close'])
    df['gravestone_doji'] = talib.CDLGRAVESTONEDOJI(df['open'], df['high'], df['low'], df['close'])
    df['long_legged_doji'] = talib.CDLLONGLEGGEDDOJI(df['open'], df['high'], df['low'], df['close'])
    df['marubozu'] = talib.CDLMARUBOZU(df['open'], df['high'], df['low'], df['close'])
    df['spinning_top'] = talib.CDLSPINNINGTOP(df['open'], df['high'], df['low'], df['close'])
    df['takuri'] = talib.CDLTAKURI(df['open'], df['high'], df['low'], df['close'])
    df['rickshawman'] = talib.CDLRICKSHAWMAN(df['open'], df['high'], df['low'], df['close'])
    df['engulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])

    # Previous 3 candles
    prev_3 = df.iloc[-5]
    prev_2 = df.iloc[-4]
    prev_1 = df.iloc[-3]

    # Signal candle
    signal_candle = df.iloc[-2]

    # Trend setup
    three_bearish = all([c['close'] < c['open'] for c in [prev_3, prev_2, prev_1]])
    three_bullish = all([c['close'] > c['open'] for c in [prev_3, prev_2, prev_1]])

    # Define bullish signal conditions
    bullish_patterns = [
        signal_candle['spinning_top'],        # neutral
        signal_candle['long_legged_doji'],    # neutral
        signal_candle['dragonfly_doji'],      # bullish
        signal_candle['hammer'],              # bullish
        signal_candle['inverted_hammer'],     # bullish
        signal_candle['takuri'],           # bullish (if white)
        signal_candle['spinning_top'],        # neutral
        signal_candle['long_legged_doji']     # neutral
    ]


    # Define bearish signal conditions
    bearish_patterns = [
        signal_candle['spinning_top'],        # neutral
        signal_candle['long_legged_doji'],    # neutral
        signal_candle['gravestone_doji'],     # bearish
        signal_candle['shooting_star'],       # bearish
        signal_candle['hanging_man'],         # bearish (if black)
        signal_candle['spinning_top'],        # neutral
        signal_candle['long_legged_doji']     # neutral
    ]


    # BUY signal
    if three_bearish and any(p > 0 for p in bullish_patterns):
        signal = "BUY"
        entry_price = df.iloc[-1]['open']

    # SELL signal
    elif three_bullish and any(p < 0 for p in bearish_patterns):
        signal = "SELL"
        entry_price = df.iloc[-1]['open']

    if signal:
        stop_loss, take_profit = predict_sl_tp(df, entry_price, signal)

    return signal, entry_price, stop_loss, take_profit

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === CALCULATE ATR (Average True Range) ===
def calculate_atr(dataframe, period=14):
    dataframe['high-low'] = dataframe['high'] - dataframe['low']
    dataframe['high-close'] = abs(dataframe['high'] - dataframe['close'].shift(1))
    dataframe['low-close'] = abs(dataframe['low'] - dataframe['close'].shift(1))
    dataframe['true_range'] = dataframe[['high-low', 'high-close', 'low-close']].max(axis=1)
    dataframe['ATR'] = dataframe['true_range'].rolling(period).mean()
    return dataframe

# === AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(dataframe, entry_price, signal):
    atr = dataframe.iloc[-1]['ATR']
    support = dataframe['low'].rolling(10).min().iloc[-1]
    resistance = dataframe['high'].rolling(10).max().iloc[-1]

    if signal == "BUY":
        stop_loss = max(support, entry_price - (atr * 0.6))
        take_profit = entry_price + ((entry_price - stop_loss) * 2)
    else:
        stop_loss = min(resistance, entry_price + (atr * 0.6))
        take_profit = entry_price - ((stop_loss - entry_price) * 2)

    return stop_loss, take_profit

# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:  # Ensure the order is not None
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: {entry_price}")
            position_id = order.order
            time.sleep(2)
            modify_request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": position_id,
                "sl": stop_loss,
                "tp": take_profit,
                "magic": MAGIC_NUMBER
            }
            modify = mt5.order_send(modify_request)
            if modify.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ SL/TP Updated: SL={stop_loss}, TP={take_profit}")
            else:
                print(f"❌ Failed to update SL/TP: {modify.retcode}")
        else:
            print(f"❌ Trade Failed: {order.retcode}")
    else:
        print("❌ Order send failed: order is None")
        
def equity_check_and_close_trades():
    global stop_trading, INITIAL_BALANCE, PROFIT_TARGET
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return

    target_equity = INITIAL_BALANCE + PROFIT_TARGET
    print(f"🎯 Monitoring for equity to reach ${target_equity:.2f}...")

    while not stop_trading:
        account_info = mt5.account_info()
        if account_info is None:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")
            wait_for_next_candle()
            continue

        equity = account_info.equity
        # print(f"🔍 Current Equity: ${equity:.2f} | Target: ${target_equity:.2f}")

        if equity >= target_equity:
            print("🚀 Equity target reached! Closing all open positions...")
            positions = mt5.positions_get()
            if positions is None or len(positions) == 0:
                print("No open positions to close.")
            else:
                for pos in positions:
                    symbol = pos.symbol
                    volume = pos.volume
                    ticket = pos.ticket
                    order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                    price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

                    close_request = {
                        "action": mt5.TRADE_ACTION_DEAL,
                        "position": ticket,
                        "symbol": symbol,
                        "volume": volume,
                        "type": order_type,
                        "price": price,
                        "deviation": 10,
                        "magic": 123456,
                        "comment": "Auto close on equity profit"
                    }

                    result = mt5.order_send(close_request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"✅ Position {ticket} closed successfully.")
                    else:
                        print(f"❌ Failed to close position {ticket}: {result.retcode}")

            stop_trading = True  # Set the stop flag
            break

        wait_for_next_candle()

def stop_at_6am_gmt():
    global stop_trading
    print("🕕 Thread to monitor 6:00 AM GMT started...")
    while not stop_trading:
        now = datetime.utcnow()
        if now.hour >= 6 and now.minute == 0:
            print("🛑 It's 6:00 AM GMT! Closing all trades and stopping the bot.")
            positions = mt5.positions_get()
            if positions:
                for pos in positions:
                    symbol = pos.symbol
                    volume = pos.volume
                    ticket = pos.ticket
                    order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                    price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

                    close_request = {
                        "action": mt5.TRADE_ACTION_DEAL,
                        "position": ticket,
                        "symbol": symbol,
                        "volume": volume,
                        "type": order_type,
                        "price": price,
                        "deviation": 10,
                        "magic": MAGIC_NUMBER,
                        "comment": "Auto close at 6AM GMT"
                    }

                    result = mt5.order_send(close_request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"✅ Position {ticket} closed successfully at 6AM GMT.")
                    else:
                        print(f"❌ Failed to close position {ticket}: {result.retcode}")
            else:
                continue

            stop_trading = True
            break

        time.sleep(30)  # check every 10 seconds

# === RUN THE BOT ===
def run_bot():
    global stop_trading
    connect_mt5()

    # Start the equity check in a separate thread
    threading.Thread(target=equity_check_and_close_trades, daemon=True).start()
    threading.Thread(target=stop_at_6am_gmt, daemon=True).start()
    
    while not stop_trading:
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            wait_for_next_candle()
            continue

        df = calculate_atr(df)
        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        # ✅ Always check and update SL to breakeven for open positions
        positions = mt5.positions_get(symbol=SYMBOL)
        if positions:
            tick = mt5.symbol_info_tick(SYMBOL)
            if tick:
                for pos in positions:
                    entry = pos.price_open
                    current_sl = pos.sl
                    current_tp = pos.tp
                    sl_distance = abs(entry - current_sl)
                    
                    if pos.type == mt5.ORDER_TYPE_BUY:
                        current_price = tick.ask
                        if current_price > entry and current_sl < entry:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": round(entry, 5),
                                "tp": current_tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ BUY SL moved to breakeven for position {pos.ticket}")
                            else:
                                print(f"❌ Failed to update BUY SL: {result.retcode}")

                    elif pos.type == mt5.ORDER_TYPE_SELL:
                        current_price = tick.bid
                        if current_price <= entry - 1.5 * sl_distance and current_sl > entry:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": round(entry, 5),
                                "tp": current_tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ SELL SL moved to breakeven for position {pos.ticket}")
                            else:
                                print(f"❌ Failed to update SELL SL: {result.retcode}")
            else:
                print("❌ Failed to fetch current price tick.")
                
        # ✅ Trade only if signal exists and allowed
        if signal:
            allow_trade = True
            if positions:
                for pos in positions:
                    if pos.type == mt5.ORDER_TYPE_BUY and signal == "BUY":
                        print("⚠️ Trade skipped: Already in a BUY position.")
                        allow_trade = False
                        break
                    elif pos.type == mt5.ORDER_TYPE_SELL and signal == "SELL":
                        print("⚠️ Trade skipped: Already in a SELL position.")
                        allow_trade = False
                        break

            if allow_trade:
                print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
                place_trade(signal, entry_price, stop_loss, take_profit)
            # else:
            #     print("⏳ Waiting for new opposite signal...")
        # else:
        #     print("⏳ No trade opportunity found.")
    
        wait_for_next_candle()
    
    print("🛑 Trading stopped due to equity target being reached.")

# Start the trading bot
run_bot()


### Retcode modification (working)

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "XAUUSDm"
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.01
MAGIC_NUMBER = 10005
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 200  # 💰 Target profit in USD
INITIAL_BALANCE = None  # 🔒 Will be set after login

# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=242864620,
        password="Password1234.",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()
        
    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

# === CHECK PROFIT AND close ALL TRADES ===
def check_profit_and_exit():
    global INITIAL_BALANCE
    account_info = mt5.account_info()
    if account_info is None:
        logging.error("❌ Could not fetch account info.")
        return

    net_profit = account_info.balance - INITIAL_BALANCE
    logging.info(f"💹 Current Net Profit: ${net_profit:.2f}")

    if net_profit >= PROFIT_TARGET:
        logging.info("🎯 Profit target reached. Closing all positions...")
        positions = mt5.positions_get()
        if positions:
            for pos in positions:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": pos.symbol,
                    "volume": pos.volume,
                    "type": mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
                    "position": pos.ticket,
                    "price": mt5.symbol_info_tick(pos.symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(pos.symbol).ask,
                    "deviation": 20,
                    "magic": MAGIC_NUMBER,
                    "comment": "close on profit target",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC
                }
                result = mt5.order_send(close_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    logging.info(f"✅ Position {pos.ticket} closed successfully.")
                else:
                    logging.warning(f"⚠️ Failed to close position {pos.ticket}: {result.retcode}")
        logging.info("🛑 Stopping bot after reaching profit target.")
        mt5.shutdown()
        exit()

# === (Rest of your existing code remains unchanged — insert check below into main loop) ===

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.df(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === CALCULATE ATR (Average True Range) ===
def calculate_atr(df, period=14):
    df['high-low'] = df['high'] - df['low']
    df['high-close'] = abs(df['high'] - df['close'].shift(1))
    df['low-close'] = abs(df['low'] - df['close'].shift(1))
    df['true_range'] = df[['high-low', 'high-close', 'low-close']].max(axis=1)
    df['ATR'] = df['true_range'].rolling(period).mean()
    return df

# === AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(df, entry_price, signal):
    atr = df.iloc[-1]['ATR']
    support = df['low'].rolling(10).min().iloc[-1]
    resistance = df['high'].rolling(10).max().iloc[-1]

    if signal == "BUY":
        stop_loss = max(support, entry_price - (atr * 1.5))
        take_profit = entry_price + ((entry_price - stop_loss) * 2)
    else:
        stop_loss = min(resistance, entry_price + (atr * 1.5))
        take_profit = entry_price - ((stop_loss - entry_price) * 2)

    return round(stop_loss, 2), round(take_profit, 2)

# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === IDENTIFY TRADE SIGNALS ===
def generate_signals(df):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None

    if df.iloc[-1]['high'] > df.iloc[-2]['high']:
        signal = "BUY"
        entry_price = df.iloc[-1]['close']
    elif df.iloc[-1]['low'] < df.iloc[-2]['low']:
        signal = "SELL"
        entry_price = df.iloc[-1]['close']

    if signal:
        stop_loss, take_profit = predict_sl_tp(df, entry_price, signal)
        log_trade_signal(signal, entry_price, stop_loss, take_profit)

    return signal, entry_price, stop_loss, take_profit

# == RETCODES ==
RETCODE_MESSAGES = {
    10004: "Requote",
    10006: "Request rejected",
    10007: "Request canceled by trader",
    10011: "Request processing error",
    10012: "Request canceled by timeout",
    10013: "Invalid request",
    10014: "Invalid volume in the request",
    10015: "Invalid price in the request",
    10016: "Invalid stops in the request",
    10017: "Trade is disabled",
    10018: "Market is closed",
    10019: "Not enough money to complete the request",
    10020: "Prices changed",
    10021: "No quotes to process the request",
    10022: "Invalid order expiration date",
    10023: "Order state changed",
    10024: "Too frequent requests",
    10025: "No changes in request",
    10026: "Autotrading disabled by server",
    10027: "Autotrading disabled by client terminal",
    10028: "Request locked for processing",
    10029: "Order or position frozen",
    10030: "Invalid order filling type",
    10031: "No connection with trade server",
    10032: "Only allowed for live accounts",
    10033: "Pending orders limit reached",
    10034: "Order/position volume limit reached for symbol",
    10035: "Incorrect or prohibited order type",
    10036: "Position already closed",
    10038: "close volume exceeds position volume",
    10039: "close order already exists for position",
    10040: "open positions limit reached",
    10041: "Activation rejected, order canceled",
    10042: "Only long positions allowed",
    10043: "Only short positions allowed",
    10044: "Only position closing allowed",
    10045: "Only FIFO closing allowed",
    10046: "Opposite positions disabled (hedging prohibited)",
}

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:  # Ensure the order is not None
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: {entry_price}")
            position_id = order.order
            time.sleep(2)
            modify_request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": position_id,
                "sl": stop_loss,
                "tp": take_profit
            }
            modify = mt5.order_send(modify_request)
            if modify.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ SL/TP Updated: SL={stop_loss}, TP={take_profit}")
            else:
                modify_message = RETCODE_MESSAGES.get(modify.retcode, str(modify.retcode))
                print(f"❌ Failed to update SL/TP: {modify_message}")
        else:
            order_message = RETCODE_MESSAGES.get(order.retcode, str(order.retcode))
            print(f"❌ Trade Failed: {order_message}")
    else:
        print("❌ Order send failed: order is None")
     
def equity_check():
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return
    global INITIAL_BALANCE
    target_equity = INITIAL_BALANCE + PROFIT_TARGET
    print(f"🎯 Monitoring for equity to reach ${target_equity:.2f}...")

    while True:
        account_info = mt5.account_info()
        if account_info is not None:
            equity = account_info.equity

            if equity >= target_equity:
                print("🚀 Equity target reached! Closing all open positions...")
                positions = mt5.positions_get()
                if positions:
                    for pos in positions:
                        symbol = pos.symbol
                        volume = pos.volume
                        ticket = pos.ticket
                        order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                        price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

                        close_request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": ticket,
                            "symbol": symbol,
                            "volume": volume,
                            "type": order_type,
                            "price": price,
                            "deviation": 10,
                            "magic": 123456,
                            "comment": "Auto close on equity profit"
                        }

                        result = mt5.order_send(close_request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"✔️ closed position {ticket}")
                        else:
                            error_message = RETCODE_MESSAGES.get(result.retcode, str(result.retcode))
                            print(f"❌ Failed to close position {ticket}, retcode: {error_message}")
        else:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")

        time.sleep(60)

# === RUN THE BOT ===
def run_bot():
    connect_mt5()
    target_profit = 200  # Target profit in USD
    initial_equity = mt5.account_info().balance  # Get initial equity of the account

    # Start the equity check in a separate thread
    threading.Thread(target=equity_check, daemon=True).start()
    
    while True:
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            time.sleep(300)
            continue

        df = calculate_atr(df)
        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        # ✅ Always check and update SL for open positions
        positions = mt5.positions_get(symbol=SYMBOL)
        if positions:
            tick = mt5.symbol_info_tick(SYMBOL)
            if tick:
                for pos in positions:
                    entry = pos.price_open
                    current_sl = pos.sl
                    sl_distance = abs(entry - current_sl)

                    # == Trailing SL for Buy position ==
                    if pos.type == mt5.ORDER_TYPE_BUY:
                        current_price = tick.ask
                        new_sl = round(current_price - sl_distance, 2)
                        if current_price > entry and new_sl > current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            error_message = RETCODE_MESSAGES.get(result.retcode, str(result.retcode)) # Converts the error codes to describtions
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ BUY SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update BUY SL: {error_message}")

                    # == Trailing SL for Sell position ==
                    elif pos.type == mt5.ORDER_TYPE_SELL:
                        current_price = tick.bid
                        new_sl = round(current_price + sl_distance, 2)
                        if current_price < entry and new_sl < current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            error_message = RETCODE_MESSAGES.get(result.retcode, str(result.retcode))
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ SELL SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update SELL SL: {error_message}")
            else:
                print("❌ Failed to fetch current price tick.")

        # ✅ Trade only if signal exists and allowed
        if signal:
            allow_trade = True
            if positions:
                for pos in positions:
                    # == Skip trade if there is already an open position of same type ==
                    if pos.type == mt5.ORDER_TYPE_BUY and signal == "BUY":
                        print("⚠️ Trade skipped: Already in a BUY position.")
                        allow_trade = False
                        break
                    elif pos.type == mt5.ORDER_TYPE_SELL and signal == "SELL":
                        print("⚠️ Trade skipped: Already in a SELL position.")
                        allow_trade = False
                        break

            if allow_trade:
                print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
                place_trade(signal, entry_price, stop_loss, take_profit)
            else:
                print("⏳ Waiting for new opposite signal...")
        else:
            print("⏳ No trade opportunity found.")

        time.sleep(300)


# Start the trading bot
run_bot()


✅ MT5 initialized and logged in successfully.
✅ Initial Balance: 730.62
🎯 Monitoring for equity to reach $930.62...
⏳ No trade opportunity found.
📢 Trade Signal: SELL | Entry: 3253.577 | SL: 3258.76 | TP: 3243.22
✅ SELL Order Placed: 3253.577
✅ SL/TP Updated: SL=3258.76, TP=3243.22
✅ SELL SL updated for position 1890910151 → 3257.3
⏳ No trade opportunity found.
✅ SELL SL updated for position 1890910151 → 3255.13
⏳ No trade opportunity found.
✅ SELL SL updated for position 1890910151 → 3252.72
⏳ No trade opportunity found.
⏳ No trade opportunity found.
📢 Trade Signal: BUY | Entry: 3254.303 | SL: 3249.0 | TP: 3264.9
✅ BUY Order Placed: 3254.303
✅ SL/TP Updated: SL=3249.0, TP=3264.9
✅ BUY SL updated for position 1891000536 → 3249.99
⚠️ Trade skipped: Already in a BUY position.
⏳ Waiting for new opposite signal...
✅ BUY SL updated for position 1891000536 → 3250.43
⏳ No trade opportunity found.
📢 Trade Signal: SELL | Entry: 3251.431 | SL: 3257.72 | TP: 3238.86
✅ SELL Order Placed: 3251.431


KeyboardInterrupt: 